In [1]:
from keras.models import Model, Sequential
from keras.layers import Input, Convolution2D, ZeroPadding2D, MaxPooling2D, Flatten, Dense, Dropout, Activation
from PIL import Image
import numpy as np
from keras.preprocessing.image import load_img, save_img, img_to_array
from keras.applications.vgg16 import preprocess_input
from keras.preprocessing import image
import matplotlib.pyplot as plt
import os
import cv2
import pafy
def loadVggFaceModel():
    model = Sequential()
    model.add(ZeroPadding2D((1,1),input_shape=(224,224, 3)))
    model.add(Convolution2D(64, (3, 3), activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(64, (3, 3), activation='relu'))
    model.add(MaxPooling2D((2,2), strides=(2,2)))

    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(128, (3, 3), activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(128, (3, 3), activation='relu'))
    model.add(MaxPooling2D((2,2), strides=(2,2)))

    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(256, (3, 3), activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(256, (3, 3), activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(256, (3, 3), activation='relu'))
    model.add(MaxPooling2D((2,2), strides=(2,2)))

    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(512, (3, 3), activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(512, (3, 3), activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(512, (3, 3), activation='relu'))
    model.add(MaxPooling2D((2,2), strides=(2,2)))

    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(512, (3, 3), activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(512, (3, 3), activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(512, (3, 3), activation='relu'))
    model.add(MaxPooling2D((2,2), strides=(2,2)))

    model.add(Convolution2D(4096, (7, 7), activation='relu'))
    model.add(Dropout(0.5))
    model.add(Convolution2D(4096, (1, 1), activation='relu'))
    model.add(Dropout(0.5))
    model.add(Convolution2D(2622, (1, 1)))
    model.add(Flatten())
    model.add(Activation('softmax'))
    
    from keras.models import model_from_json
    model.load_weights('D:\\opencv\\vgg_face_weights.h5')

    vgg_face_descriptor = Model(inputs=model.layers[0].input, outputs=model.layers[-2].output)

    return vgg_face_descriptor
def preprocess_image(image_path):
    img = load_img(image_path, target_size=(224, 224))
    img = img_to_array(img)
    img = np.expand_dims(img, axis=0)
    img = preprocess_input(img)
    return img

model = loadVggFaceModel()
images_path=input('Please Enter the inpur images path here: ')

persons = dict()
for file in os.listdir(images_path):
    person, extension = file.split(".")
    persons[person] = model.predict(preprocess_image(images_path+'\\%s.jpg' % (person)))[0,:]

print("person representations retrieved successfully")

def findCosineSimilarity(train, test):
    a = np.matmul(np.transpose(train), test)
    b = np.sum(np.multiply(train, train))
    c = np.sum(np.multiply(test, test))
    return 1 - (a / (np.sqrt(b) * np.sqrt(c)))

def findEuclideanDistance(train, test):
    euclidean_distance = train - test
    euclidean_distance = np.sum(np.multiply(euclidean_distance, euclidean_distance))
    euclidean_distance = np.sqrt(euclidean_distance)
    return euclidean_distance

from mtcnn.mtcnn import MTCNN
detector=MTCNN()

import urllib
from urllib import request as ur

from datetime import datetime
faceFound = []
ts = [0,0,False,' ']


#"from datetime import datetime
# Capture time duration of faces detected
faceFound = {}
ts = [0,0,False,' ']
print("dictionary created")


# In[21]:


video_capture=cv2.VideoCapture('video_home.mp4')
cap = cv2.VideoCapture('john-nikki.mp4')

# In[22]:


while True:
    _,img=video_capture.read()
    result=detector.detect_faces(img)
"


url = "https://www.youtube.com/watch?v=HOi4StHiYaw"
videoPafy = pafy.new(url)
best = videoPafy.getbest()

cap=cv2.VideoCapture(best.url)
#for webcam
#cap=cv2.VideoCapture(0)


while True:
    _,img=cap.read()

    result=detector.detect_faces(img)
# in the result we will have multiple number of dictionaries for each image, and we need to extract the bounding box of the face.      
    if len(result):
        if ts[2] == False:
            ts[0] = datetime.now()
            ts[2] = True
        for i in range(len(result)):
            (startX,startY,endX,endY)=(result[i]["box"][0], result[i]["box"][1],result[i]["box"][0]+result[i]["box"][2], result[i]["box"][1] + result[i]["box"][3])
            confidence = result[i]["confidence"]
            if (confidence > 0.5):
                cv2.rectangle(img, (startX, startY), (endX, endY), (0, 255, 0), 2)
                if (startY >=0) and (endY >=0) and (startX >=0) and (endX >=0):
                    frame_face = img[startY:endY, startX:endX]
                    frame_face=cv2.resize(frame_face,(224,224))
            crop_img=img_to_array(frame_face)
            crop_img=np.expand_dims(crop_img,axis=0)
            crop_img=preprocess_input(crop_img)
            test=model.predict(crop_img)[0,:]  
            
            found = 0
            for i in persons:
                label_name = i
                train = persons[i]
                #similarity = findEuclideanDistance(train,test)
                similarity = findCosineSimilarity(train, test)
                print(similarity)
                #if(similarity < 100):
                if(similarity < 0.5):
                    #cv2.rectangle(img, (startX, startY), (endX, endY), (0, 255, 0), 2)
                    img=cv2.putText(img,label_name,(startX,startY),cv2.FONT_HERSHEY_SIMPLEX,1,(255,0,255),2,cv2.LINE_AA)
                    #img=cv2.putText(img,str(np.max(person)),(x2,y2+10),cv2.FONT_HERSHEY_SIMPLEX,0.5,(0,0,255),1,cv2.LINE_AA)\
                    ts[3] = label_name
                    found=1
                    break

            if (found==0):
                img=cv2.putText(img,'Unknown',(startX,startY),cv2.FONT_HERSHEY_SIMPLEX,1,(255,0,255),2,cv2.LINE_AA)
                ts[3] = 'Unknown'
            #cv2.imshow('Video',img)
            
    else:
        if ts[2] == True:
            ts[1] = datetime.now()
            ts[2] = False
            diff = ts[1]-ts[0]
            if diff.seconds!=0:
                faceFound.append((ts[3], diff.seconds+1, ts[0].strftime("%m/%d/%Y %H:%M:%S"), ts[1].strftime("%m/%d/%Y %H:%M:%S")))
        
    cv2.imshow('video',img)
    if cv2.waitKey(1)==27:
        break
cap.release()
cv2.destroyAllWindows()

print(faceFound)

import pandas as pd
time_log=pd.DataFrame(faceFound,columns = ('Person Name','total time(sec)','entry time','exit time'))

print(time_log)

Using TensorFlow backend.


D:\MySQL\image\celebs\oneshot\Naved

D:\MySQL\image\celebs\oneshot\youtube